In [1]:
import tweepy
import csv
import json
import os

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
#Add your credentials here
twitter_keys = {
        'consumer_key':        'Z8slabsivDl8YMskkKXHn6Rpk',
        'consumer_secret':     'Liwia9OD2WS5QximVR3AzN1TM0vdmLejwBLr01GlUsENvgYVXY',
        'access_token_key':    '80337313-DrhQUb770oFY0vlqL8w8PjMoF4cpOUthwjq310y7Q',
        'access_token_secret': 'AKHif2AGfvvhmuZUHq5l7cz26b1yZoMycrhxlcsMcEaZ9'
    }

# Samples using tweepy

In [3]:
#Setup access to API
auth = tweepy.OAuthHandler(twitter_keys['consumer_key'], twitter_keys['consumer_secret'])
auth.set_access_token(twitter_keys['access_token_key'], twitter_keys['access_token_secret'])

api = tweepy.API(auth)

#Make call on home timeline, print each tweets text
public_tweets = api.home_timeline()
for tweet in public_tweets:
    print(tweet.text)

RT @lmeyerov: Spent the weekend writing unit tests for dataframe non-determinism / weird indexes / coercions after upgrading pandas+rapids…
BREAKING: Car rams into a crowd during a parade in #Volkmarsen, Germany, more than 10 people injured - reports… https://t.co/uL4sS23SsX
Metamorfo: the banking Trojan with a long list of tricks: Metamorfo is a banking Trojan, first discovered… https://t.co/56DbtGyLGe #infosec
How to Scope Your Organization for NERC CIP: Knowing how to scope your organization for NERC CIP security… https://t.co/jqHyuk6XzM #infosec
Latest Security News from RSAC 2020: Check out Dark Reading’s updated, exclusive coverage of the news and… https://t.co/nxBbnbmnnB #infosec
EU plans to link member states’ facial recognition databases: EU plans to link member states’ facial… https://t.co/sEb2JssUlq #infosec
WhatsApp security alert: Strangers could get an invitation to your private group: WhatsApp security alert… https://t.co/DN5rpCQ2Xb #infosec
Raccoon: The Story of a Typic

In [4]:
import json
# https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object
status = public_tweets[0]
json_str = json.dumps(status._json, indent=4, sort_keys=True)
print(json_str)

{
    "contributors": null,
    "coordinates": null,
    "created_at": "Mon Feb 24 14:40:37 +0000 2020",
    "entities": {
        "hashtags": [],
        "symbols": [],
        "urls": [],
        "user_mentions": [
            {
                "id": 15315247,
                "id_str": "15315247",
                "indices": [
                    3,
                    12
                ],
                "name": "lmeyerov",
                "screen_name": "lmeyerov"
            }
        ]
    },
    "favorite_count": 0,
    "favorited": false,
    "geo": null,
    "id": 1231952071579951104,
    "id_str": "1231952071579951104",
    "in_reply_to_screen_name": null,
    "in_reply_to_status_id": null,
    "in_reply_to_status_id_str": null,
    "in_reply_to_user_id": null,
    "in_reply_to_user_id_str": null,
    "is_quote_status": false,
    "lang": "en",
    "place": null,
    "retweet_count": 0,
    "retweeted": false,
    "retweeted_status": {
        "contributors": null,
        "c

In [5]:
item = []
json_obj = json.loads(json_str)

if json_obj.get('retweeted_status'):
    print('Is retweeted post')
else:
    print('Is orignial post')

# Tweet information
item.append(json_obj['id']) # The integer representation of the unique identifier for this Tweet
item.append(json_obj['created_at']) # UTC time when this Tweet was created.
item.append(json_obj['favorite_count']) # Nullable. Indicates approximately how many times this Tweet has been liked by Twitter users
item.append(json_obj['retweet_count']) # Number of times this Tweet has been retweeted.
item.append(json_obj['text']) # Actual text posted
if json_obj.get('extended_tweet'):
    item.append(json_obj['extended_tweet']['full_text']) # full text when over 280 chars limit    
else:
    item.append(None)
item.append(json_obj['source']) # Utility used to post the Tweet
item.append(json_obj['coordinates']) # Nullable. Represents the geographic location of this Tweet as reported by the user or client application.

# User information
item.append(json_obj['user']['screen_name']) # user who post this
item.append(json_obj['user']['created_at']) # user account creation date
item.append(json_obj['user']['followers_count']) # The number of followers this account currently has.
item.append(json_obj['user']['verified']) # When true, indicates that the user has a verified account.

item

Is retweeted post


[1231952071579951104,
 'Mon Feb 24 14:40:37 +0000 2020',
 0,
 0,
 'RT @lmeyerov: Spent the weekend writing unit tests for dataframe non-determinism / weird indexes / coercions after upgrading pandas+rapids…',
 None,
 '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
 None,
 'jeremyphoward',
 'Fri Aug 06 04:58:18 +0000 2010',
 87064,
 False]

# Streaming tweets

In [6]:
class MyStreamListener(tweepy.StreamListener):           
    def __init__(self, api=None):
        super().__init__()
        self.accept_retweet = False
        self.stop_count = 30000
        self.count = 0
        
    def write_csv(self, data):
        if not os.path.exists('tweets.csv'):
            with open('tweets.csv', 'a') as outfile:
                writer = csv.writer(outfile)
                cols = ['id', 'created_at', 'favorite_count', 'retweet_count', 'text', 'extended_tweet',
                       'source', 'coordinates', 'screen_name', 'created_at', 'followers_count', 'verified']
                writer.writerow(cols)
        with open('tweets.csv', 'a') as outfile:
            writer = csv.writer(outfile)
            writer.writerow(data)
            
    def on_status(self, status):        
        try:
            if self.count >= self.stop_count: # stop streaming
                return False

            json_str = json.dumps(status._json, indent=4, sort_keys=True)        
            json_obj = json.loads(json_str)

            if json_obj.get('retweeted_status'):
                isRetweet = True
            else:
                isRetweet = False

            # Do we accept retweet or not
            if isRetweet == self.accept_retweet:             
                item = []
                # Tweet information
                item.append(json_obj['id']) # The integer representation of the unique identifier for this Tweet
                item.append(json_obj['created_at']) # UTC time when this Tweet was created.
                item.append(json_obj['favorite_count']) # Nullable. Indicates approximately how many times this Tweet has been liked by Twitter users
                item.append(json_obj['retweet_count']) # Number of times this Tweet has been retweeted.
                item.append(json_obj['text']) # Actual text posted
                if json_obj.get('extended_tweet'):
                    item.append(json_obj['extended_tweet']['full_text']) # full text when over 280 chars limit    
                else:
                    item.append(None)
                item.append(json_obj['source']) # Utility used to post the Tweet
                item.append(json_obj['coordinates']) # Nullable. Represents the geographic location of this Tweet as reported by the user or client application.

                # User information
                item.append(json_obj['user']['screen_name']) # user who post this
                item.append(json_obj['user']['created_at']) # user account creation date
                item.append(json_obj['user']['followers_count']) # The number of followers this account currently has.
                item.append(json_obj['user']['verified']) # When true, indicates that the user has a verified account.        
                self.write_csv(item)
                # print(json_obj['text'])
                self.count += 1
                if self.count % 1000 == 0:
                    print(self.count)
        except Exception as e:
            print(str(ex))
            
    def on_error(self, status_code):
        if status_code == 420:
            return False

In [7]:
auth = tweepy.OAuthHandler(twitter_keys['consumer_key'], twitter_keys['consumer_secret'])
auth.set_access_token(twitter_keys['access_token_key'], twitter_keys['access_token_secret'])
api = tweepy.API(auth)

myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)

track_terms = ['covid19', 'wuhanvirus', 'Coronavirus', 'COVIDー19',
               'infected', '2019nCoV', 'WuhanOutbreak', 'Singapore']
languages = ['en']
region = [103.566667,1.154266,104.093392,1.485141]
myStream.filter(track=track_terms, locations=region, languages=languages,
                filter_level='low', is_async=True)

In [8]:

def clean_tweet(self, tweet): 
    ''' 
    Use sumple regex statemnents to clean tweet text by removing links and special characters
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) \
                                |(\w+:\/\/\S+)", " ", tweet).split()) 
def deEmojify(text):
    '''
    Strip all non-ASCII characters to remove emoji characters
    '''
    if text:
        return text.encode('ascii', 'ignore').decode('ascii')
    else:
        return None

In [9]:
# Import package
import tweepy
import json

class MyStreamListener(tweepy.StreamListener):
    def __init__(self, api=None):
        super(MyStreamListener, self).__init__()
        self.num_tweets = 0
        self.file = open("tweets.txt", "w")
    def on_status(self, status):
        tweet = status._json
        self.file.write(json.dumps(tweet) + '\n')
        tweet_list.append(status)
        self.num_tweets += 1
        if self.num_tweets < 10:
            return True
        else:
            return False
        self.file.close()

# Store OAuth authentication credentials in relevant variables (these are DataCamp tokens)
access_token = "1092294848-aHN7DcRP9B4VMTQIhwqOYiB14YkW92fFO8k8EPy"
access_token_secret = "X4dHmhPfaksHcQ7SCbmZa2oYBBVSD2g8uIHXsp5CTaksx"
consumer_key = "nZ6EA0FxZ293SxGNg8g8aP0HM"
consumer_secret = "fJGEodwe3KiKUnsYJC3VRndj7jevVvXbK2D5EiJ2nehafRgA6i"

# Pass OAuth details to tweepy's OAuth handler
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Initialize Stream listener
l = MyStreamListener()

# Create you Stream object with authentication
stream = tweepy.Stream(auth, l)


# Filter Twitter Streams to capture data by the keywords:
stream.filter(track=['covid19', 'wuhanvirus', 'Coronavirus', 'COVIDー19',
               'infected', '2019nCoV', 'WuhanOutbreak', 'Singapore'])

In [10]:
stream

In [11]:
# Import package
import pandas as pd

# Build DataFrame of tweet texts and languages
df = pd.DataFrame(tweets_data, columns=['text', 'lang'])

# Print head of DataFrame
print(df.head())

Exception in thread Thread-6:
Traceback (most recent call last):
  File "<ipython-input-6-5b401450e1ce>", line 53, in on_status
    self.write_csv(item)
  File "<ipython-input-6-5b401450e1ce>", line 17, in write_csv
    writer.writerow(data)
  File "c:\users\65848\appdata\local\programs\python\python36\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode characters in position 56-57: character maps to <undefined>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\users\65848\appdata\local\programs\python\python36\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "c:\users\65848\appdata\local\programs\python\python36\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "c:\users\65848\appdata\local\programs\python\python36\lib\site-packages\tweepy\streaming.

NameError: name 'tweets_data' is not defined

In [ ]:
# Import package
import tweepy

# Store OAuth authentication credentials in relevant variables (these are DataCamp tokens)
access_token = "1092294848-aHN7DcRP9B4VMTQIhwqOYiB14YkW92fFO8k8EPy"
access_token_secret = "X4dHmhPfaksHcQ7SCbmZa2oYBBVSD2g8uIHXsp5CTaksx"
consumer_key = "nZ6EA0FxZ293SxGNg8g8aP0HM"
consumer_secret = "fJGEodwe3KiKUnsYJC3VRndj7jevVvXbK2D5EiJ2nehafRgA6i"

# Pass OAuth details to tweepy's OAuth handler
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)